### Ex utero embryos metacell construction

In [1]:
setwd(here::here())

In [2]:
library("metacell")

In [3]:
scdb_init("scrna_db/")

initializing scdb to scrna_db/



In [4]:
source("scripts/generic_mc.r")
tgconfig::override_params(config_file = "config/emb_exe.yaml",package = "metacell")

In [5]:
scfigs_init("figs")

In [6]:
bad_genes = read.table("data/exutero_f.bad_genes.txt",sep = "\t",stringsAsFactors = F)
bad_genes = bad_genes[,1]

In [7]:
mat_nm = "exutero_f"

generate_mc(mat_nm, color_key=NA,add_bad_genes = bad_genes,recompute = T)

Calculating gene statistics... 

will downsamp

done downsamp

will gen mat_n

done gen mat_n

done computing basic gstat, will compute trends



..done


will downsample the matrix, N= 3034

will build balanced knn graph on 12110 cells and 829 genes, this can be a bit heavy for >20,000 cells

running bootstrap to generate cocluster

done resampling

filtered 4588169 left with 797614 based on co-cluster imbalance

building metacell object, #mc 151

add batch counts

compute footprints

compute absolute ps

compute coverage ps

reordering metacells by hclust and most variable two markers

reorder on Rhox5 vs Pou5f1

starting split outliers 

add batch counts

compute footprints

compute absolute ps

compute coverage ps



#### Next color metacells by projecting them on the WT atlas

In [8]:
source("scripts/atlas_proj.r")

In [9]:
mat_exutero = scdb_mat("exutero_f")
mc_exutero = scdb_mc("exutero_f_bs500f")
mat_wt = scdb_mat("embexe")
mc_wt = scdb_mc("embexe_recolored")
gset_wt = scdb_gset("embexe")
gset_exutero = scdb_gset("exutero_f")

col_to_ct = mc_wt@color_key$group
names(col_to_ct) = mc_wt@color_key$color
ct_to_col = mc_wt@color_key$color
names(ct_to_col) = mc_wt@color_key$group

In [12]:
#common_genes = intersect(intersect(rownames(mat_wt@mat),rownames(mat_gotg@mat)),rownames(mat_exutero@mat))
common_genes = intersect(rownames(mc_wt@e_gc),rownames(mc_exutero@e_gc))
feat_genes = intersect(names(gset_wt@gene_set),common_genes)

In [14]:
exutero_to_wt_projection = project_query_mc_egc_on_reference_mc_egc(egc_query = mc_exutero@e_gc[feat_genes,],
                                                              egc_reference = mc_wt@e_gc[feat_genes,])
exutero_projected_cell_type = mc_wt@colors[exutero_to_wt_projection$best_reference_metacell]

In [15]:
length(exutero_projected_cell_type)
ncol(mc_exutero@e_gc)

[1] 151

[1] 151

In [16]:
mc_exutero@colors = exutero_projected_cell_type

In [17]:
table(exutero_projected_cell_type)

exutero_projected_cell_type
#0F4A9C #19ce5b #1A1A1A #354E23 #45d1c5 #532C8A #53f1fc #635547 #649146 #67000d 
      1       3       5       1       1       1       3      15       1       7 
#7F6874 #824A09 #8870ad #8DB5CE #8e7e59 #90BF75 #9e6762 #C594BF #D96B2B #DABE99 
     19       4       5       1       1       4       1      15       2       6 
#DFCDE4 #EF5A9D #F25CD4 #F397C0 #F6BFCB #FACB12 #FBBE92 #FF5608 #b2b2b2 #c19f70 
      1       1       1       1       2       1       1       6       2       2 
#c9a997 #cc7818 #d6616b #f7f79e #ff891c #ffad1e #ffd854 
      1       3       6       2       1       6      18 

In [18]:
scdb_add_mc(id = "exutero_f",mc = mc_exutero)